In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!unzip -u /kaggle/input/facial-keypoints-detection/training.zip
!unzip -u /kaggle/input/facial-keypoints-detection/test.zip

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
idlookup_df = pd.read_csv("/kaggle/input/facial-keypoints-detection/IdLookupTable.csv")
train_df = pd.read_csv("/kaggle/working/training.csv")
test_df = pd.read_csv("/kaggle/working/test.csv")

In [ ]:
idlookup_df.head()

In [ ]:
train_df.head()

In [ ]:
test_df.head()

In [ ]:
train_df.shape

In [ ]:
train_df.duplicated().sum()

In [ ]:
train_df.isnull().sum()

In [ ]:
unclean_train_df = train_df.fillna(method = 'ffill')
print("unclean_train_data shape: {}\n".format(np.shape(unclean_train_df)))

In [ ]:
train_df.dropna(inplace=True)
train_df.shape

In [ ]:
print("Number of missing pixel values: {}".format(len(unclean_train_df) - int(train_df.Image.apply(lambda x: len(x.split())).value_counts().values)))

In [ ]:
img_height = 96
img_width = 96

In [ ]:
def plot_sample(image, keypoint, axis, title):
    image = image.reshape(img_height, img_width)
    axis.imshow(image, cmap="gray")
    axis.scatter(keypoint[::2], keypoint[1::2], marker='x', s=20)
    plt.title(title)

In [ ]:
def convert_data_to_image(image_data):
    images = []
    for _, sample in image_data.iterrows():
        image = np.array(sample["Image"].split(' '), dtype=int)
        image = np.reshape(image, (img_height,img_width,1))
        images.append(image)
    images = np.array(images)/255
    return images

In [ ]:
def get_keypoints_features(keypoint_data):
    keypoint_data = keypoint_data.drop("Image", axis=1)
    keypoint_features = []
    for _, sample_keypoints in keypoint_data.iterrows():
        keypoint_features.append(sample_keypoints)
    
    keypoint_features = np.array(keypoint_features, dtype="float")
    return keypoint_features

In [ ]:
sample_image_index = 10

train_images = convert_data_to_image(train_df)

train_keypoints = get_keypoints_features(train_df)

print("Shape of train_images: {}".format(np.shape(train_images)))
print("Shape of train_keypoints: {}".format(np.shape(train_keypoints)))

fig, axis = plt.subplots()
plot_sample(train_images[sample_image_index], train_keypoints[sample_image_index], axis, "Sample image & keypoints")

In [ ]:
sample_image_index = 10

unclean_train_images = convert_data_to_image(unclean_train_df)

unclean_train_keypoints = get_keypoints_features(unclean_train_df)

print("Shape of unclean_train_images: {}".format(np.shape(unclean_train_images)))
print("Shape of unclean_train_keypoints: {}".format(np.shape(unclean_train_keypoints)))

fig, axis = plt.subplots()
plot_sample(unclean_train_images[sample_image_index], unclean_train_keypoints[sample_image_index], axis, "Sample image & keypoints")

In [ ]:
test_images = convert_data_to_image(test_df)

test_keypoints = get_keypoints_features(test_df)

print("Shape of test_images: {}".format(np.shape(test_images)))
print("Shape of test_keypoints: {}".format(np.shape(test_keypoints)))

In [ ]:
full_train_images = train_images
full_train_keypoints = train_keypoints

full_train_images = np.concatenate((full_train_images, unclean_train_images))
full_train_keypoints = np.concatenate((full_train_keypoints, unclean_train_keypoints))

In [ ]:
#train_images = unclean_train_images
#train_keypoints = unclean_train_keypoints

In [ ]:
def left_right_flip(images, keypoints_features):
    flipped_keypoints = []
    flipped_images = np.flip(images, axis=2)
    for index, sample_keypoints in enumerate(keypoints_features):
        flipped_keypoints.append([96.-coord if index%2==0 else coord for index,coord in enumerate(sample_keypoints)])
        
    return flipped_images, flipped_keypoints

In [ ]:
horizontal_flip = True

flipped_train_images = np.array([])
flipped_train_keypoints = np.array([])

if horizontal_flip:
    flipped_train_images, flipped_train_keypoints = left_right_flip(train_images, train_keypoints)
    print("Shape of flipped_train_images: {}".format(np.shape(flipped_train_images)))
    print("Shape of flipped_train_keypoints: {}".format(np.shape(flipped_train_keypoints)))
    
    full_train_images = np.concatenate((full_train_images, flipped_train_images))
    full_train_keypoints = np.concatenate((full_train_keypoints, flipped_train_keypoints))
    
    fig, axis = plt.subplots()
    plot_sample(flipped_train_images[sample_image_index], flipped_train_keypoints[sample_image_index], axis, "Horizontally Flipped")
    
    print(full_train_images.shape)
    print(full_train_keypoints.shape)

In [ ]:
import cv2
import math

In [ ]:
def rotate_augmentation(images, keypoints_features, rotation_angles):
    rotated_images = []
    rotated_keypoints_features = []
    for angle in rotation_angles:    
        for angle in [angle,-angle]:
            M = cv2.getRotationMatrix2D((img_height/2,img_width/2), angle, 1.0)
            angle_rad = -angle*math.pi/180
            
            
            for image in images:
                rotated_image = cv2.warpAffine(image, M, (img_height,img_width), flags=cv2.INTER_CUBIC)
                rotated_images.append(rotated_image)

            for keypoint in keypoints_features:
                rotated_keypoint = keypoint - img_height/2.    
                
                for idx in range(0,len(rotated_keypoint),2):
                    rotated_keypoint[idx] = rotated_keypoint[idx]*math.cos(angle_rad)-rotated_keypoint[idx+1]*math.sin(angle_rad)
                    rotated_keypoint[idx+1] = rotated_keypoint[idx]*math.sin(angle_rad)+rotated_keypoint[idx+1]*math.cos(angle_rad)
                    
                rotated_keypoint += img_height/2
                rotated_keypoints_features.append(rotated_keypoint)
            
    return np.reshape(rotated_images,(-1,img_height,img_width,1)), rotated_keypoints_features

In [ ]:
rotation_augmentation = True
rotation_angles = [15] 

rotated_train_images = np.array([])
rotated_train_keypoints_features = np.array([])

if rotation_augmentation:
    rotated_train_images, rotated_train_keypoints_features = rotate_augmentation(train_images, train_keypoints, rotation_angles)
    print("Shape of rotated_train_images: {}".format(np.shape(rotated_train_images)))
    print("Shape of rotated_train_keypoints: {}".format(np.shape(rotated_train_keypoints_features)))
    
    full_train_images = np.concatenate((full_train_images, rotated_train_images))
    full_train_keypoints = np.concatenate((full_train_keypoints, rotated_train_keypoints_features))
    
    fig, axis = plt.subplots()
    plot_sample(rotated_train_images[sample_image_index], rotated_train_keypoints_features[sample_image_index], axis, "Rotation Augmentation")
    
    print(full_train_images.shape)
    print(full_train_keypoints.shape)

In [ ]:
def alter_brightness(images, keypoints):    
    altered_brightness_images = []
    inc_brightness_images = np.clip(images*1.2, 0.0, 1.0)    
    dec_brightness_images = np.clip(images*0.6, 0.0, 1.0)   
    
    altered_brightness_images.extend(inc_brightness_images)
    altered_brightness_images.extend(dec_brightness_images)
    
    return altered_brightness_images, np.concatenate((keypoints, keypoints))

In [ ]:
brightness_augmentation = True

altered_brightness_train_images = np.array([])
altered_brightness_train_keypoints = np.array([])

if brightness_augmentation:
    altered_brightness_train_images, altered_brightness_train_keypoints = alter_brightness(train_images, train_keypoints)
    print(f"Shape of altered_brightness_train_images: {np.shape(altered_brightness_train_images)}")
    print(f"Shape of altered_brightness_train_keypoints: {np.shape(altered_brightness_train_keypoints)}")
    
    
    full_train_images = np.concatenate((full_train_images, altered_brightness_train_images))
    full_train_keypoints = np.concatenate((full_train_keypoints, altered_brightness_train_keypoints))
    
    fig, axis = plt.subplots()
    plot_sample(altered_brightness_train_images[sample_image_index], altered_brightness_train_keypoints[sample_image_index], axis, "Increased Brightness") 
    
    fig, axis = plt.subplots()
    image_plot =  altered_brightness_train_images[len(altered_brightness_train_images)//2+sample_image_index]
    keypoints_plot = altered_brightness_train_keypoints[len(altered_brightness_train_images)//2+sample_image_index]
    plot_sample(image_plot, keypoints_plot, axis, "Decreased Brightness") 
    
    print(full_train_images.shape)
    print(full_train_keypoints.shape)

In [ ]:
def shift_images(images, keypoints, pixel_shifts):
    shifted_images = []
    shifted_keypoints = []
    for shift in pixel_shifts:    
        for (shift_x,shift_y) in [(-shift,-shift),(-shift,shift),(shift,-shift),(shift,shift)]:
            matrix = np.float32([[1,0,shift_x],[0,1,shift_y]])
            
            for image, keypoint in zip(images, keypoints):
                shifted_image = cv2.warpAffine(image, matrix, (img_height,img_width), flags=cv2.INTER_CUBIC)
                shifted_keypoint = np.array([(point+shift_x) if idx%2==0 else (point+shift_y) for idx, point in enumerate(keypoint)])
                
                if np.all(0.0<shifted_keypoint) and np.all(shifted_keypoint<img_height):
                    shifted_images.append(shifted_image.reshape(img_height,img_width,1))
                    shifted_keypoints.append(shifted_keypoint)
                    
    shifted_keypoints = np.clip(shifted_keypoints,0.0,img_height)
    return shifted_images, shifted_keypoints

In [ ]:
shift_augmentation = True
pixel_shifts = [12] 

if shift_augmentation:
    shifted_train_images, shifted_train_keypoints = shift_images(train_images, train_keypoints, pixel_shifts)
    print(f"Shape of shifted_train_images: {np.shape(shifted_train_images)}")
    print(f"Shape of shifted_train_keypoints: {np.shape(shifted_train_keypoints)}")
    
    full_train_images = np.concatenate((full_train_images, shifted_train_images))
    full_train_keypoints = np.concatenate((full_train_keypoints, shifted_train_keypoints))

    fig, axis = plt.subplots()
    plot_sample(shifted_train_images[sample_image_index], shifted_train_keypoints[sample_image_index], axis, "Shift Augmentation")
    
    print(full_train_images.shape)
    print(full_train_keypoints.shape)

In [ ]:
def add_noise(images):
    noisy_images = []
    for image in images:
        noisy_image = cv2.add(image, 0.008*np.random.randn(img_height, img_width, 1))
        noisy_images.append(noisy_image.reshape(img_height, img_width, 1))
        
    return noisy_images

In [ ]:
random_noise_augmentation = True

if random_noise_augmentation:
    noisy_train_images = add_noise(train_images)
    
    print(f"Shape of noisy_train_images: {np.shape(noisy_train_images)}")
    
    full_train_images = np.concatenate((full_train_images, noisy_train_images))
    full_train_keypoints = np.concatenate((full_train_keypoints, train_keypoints))
   
    fig, axis = plt.subplots()
    plot_sample(noisy_train_images[sample_image_index], train_keypoints[sample_image_index], axis, "Random Noise Augmentation")
    
    print(full_train_images.shape)
    print(full_train_keypoints.shape)

In [ ]:
fig = plt.figure(figsize=(20,8))
for i in range(10):
    axis = fig.add_subplot(2, 5, i+1, xticks=[], yticks=[])
    plot_sample(train_images[i], train_keypoints[i], axis, "")
plt.show()

In [ ]:
if horizontal_flip:
    print("Horizontal Flip Augmentation: ")
    fig = plt.figure(figsize=(20,8))
    for i in range(10):
        axis = fig.add_subplot(2,5,i+1,xticks=[],yticks=[])
        plot_sample(flipped_train_images[i], flipped_train_keypoints[i], axis, "")
    plt.show()

In [ ]:
if rotation_augmentation:
    print("Rotation Augmentation: ")
    fig = plt.figure(figsize=(20,8))
    for i in range(10):
        axis = fig.add_subplot(2, 5, i+1, xticks=[], yticks=[])
        plot_sample(rotated_train_images[i], rotated_train_keypoints_features[i], axis, "")
    plt.show()

In [ ]:
if brightness_augmentation:
    print("Brightness Augmentation: ")
    fig = plt.figure(figsize=(20,8))
    for i in range(10):
        axis = fig.add_subplot(2, 5, i+1, xticks=[], yticks=[])
        plot_sample(altered_brightness_train_images[i], altered_brightness_train_keypoints[i], axis, "")
    plt.show()

In [ ]:
if shift_augmentation:
    print("Shift Augmentation: ")
    fig = plt.figure(figsize=(20,8))
    for i in range(10):
        axis = fig.add_subplot(2, 5, i+1, xticks=[], yticks=[])
        plot_sample(shifted_train_images[i], shifted_train_keypoints[i], axis, "")
    plt.show()

In [ ]:
if random_noise_augmentation:
    print("Random Noise Augmentation: ")
    fig = plt.figure(figsize=(20,8))
    for i in range(10):
        axis = fig.add_subplot(2, 5, i+1, xticks=[], yticks=[])
        plot_sample(noisy_train_images[i], train_keypoints[i], axis, "")
    plt.show()

In [ ]:
from sklearn.utils import shuffle
full_train_images, full_train_keypoints = shuffle(full_train_images, full_train_keypoints, random_state=0)
print(full_train_images.shape)
print(full_train_keypoints.shape)

In [ ]:
from keras.models import Sequential, Model, load_model
from keras.layers import Activation, Convolution2D, MaxPooling2D, BatchNormalization, Flatten, Dense, Dropout, Conv2D, MaxPool2D, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, EarlyStopping
from keras.optimizers import Adam

In [ ]:
model = Sequential()

# ----------------------------------------
model.add(Convolution2D(32, (3,3), padding='same', use_bias=False, input_shape=(96,96,1)))
model.add(LeakyReLU(alpha = 0.1))
model.add(BatchNormalization())

model.add(Convolution2D(32, (3,3), padding='same', use_bias=False))
model.add(LeakyReLU(alpha = 0.1))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2, 2)))


# ----------------------------------------
model.add(Convolution2D(64, (3,3), padding='same', use_bias=False))
model.add(LeakyReLU(alpha = 0.1))
model.add(BatchNormalization())

model.add(Convolution2D(64, (3,3), padding='same', use_bias=False))
model.add(LeakyReLU(alpha = 0.1))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2, 2)))


# ----------------------------------------
model.add(Convolution2D(96, (3,3), padding='same', use_bias=False))
model.add(LeakyReLU(alpha = 0.1))
model.add(BatchNormalization())

model.add(Convolution2D(96, (3,3), padding='same', use_bias=False))
model.add(LeakyReLU(alpha = 0.1))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2, 2)))


# ----------------------------------------
model.add(Convolution2D(128, (3,3),padding='same', use_bias=False))
model.add(LeakyReLU(alpha = 0.1))
model.add(BatchNormalization())

model.add(Convolution2D(128, (3,3),padding='same', use_bias=False))
model.add(LeakyReLU(alpha = 0.1))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2, 2)))


# ----------------------------------------
model.add(Convolution2D(256, (3,3),padding='same',use_bias=False))
model.add(LeakyReLU(alpha = 0.1))
model.add(BatchNormalization())

model.add(Convolution2D(256, (3,3),padding='same',use_bias=False))
model.add(LeakyReLU(alpha = 0.1))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2, 2)))


# ----------------------------------------
model.add(Convolution2D(512, (3,3), padding='same', use_bias=False))
model.add(LeakyReLU(alpha = 0.1))
model.add(BatchNormalization())

model.add(Convolution2D(512, (3,3), padding='same', use_bias=False))
model.add(LeakyReLU(alpha = 0.1))
model.add(BatchNormalization())

model.add(Flatten())
model.add(Dense(512,activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(30))
model.summary()

In [ ]:
import os

model_file_path = "best_model.hdf5"
num_epochs = 100
batch_size = 64
validation_ratio = 0.05
if False:
#if os.path.exists(model_file_path):
    model = load_model(model_file_path)

else:
    checkpoint = ModelCheckpoint(filepath=model_file_path, monitor="val_mea", verbose=1, save_best_only=True, mode="min")
    es = EarlyStopping(monitor="val_mea", mode="min", verbose=1, patience=10)
    #lr_reduce = ReduceLROnPlateau(monitor="val_mea", factor=0, min_delta=0.001, patience=1, verbose=1)
    
    model.compile(optimizer="adam", loss="mean_squared_error", metrics=['mae', 'acc'])
    
    history = model.fit(full_train_images, full_train_keypoints, epochs=num_epochs, batch_size=batch_size, validation_split=validation_ratio, callbacks=[checkpoint, es], verbose=1)

In [ ]:
plt.plot(history.history['mae'])
plt.plot(history.history['val_mae'])
plt.title('Mean Absolute Error vs Epoch')
plt.ylabel('Mean Absolute Error')
plt.xlabel('Epochs')
plt.legend(['train', 'validation'], loc='upper right')
plt.show()

In [ ]:
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('Accuracy vs Epoch')
plt.ylabel('Accuracy')
plt.xlabel('Epochs')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Loss vs Epoch')
plt.ylabel('Loss')
plt.xlabel('Epochs')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

In [ ]:
%%time
 
#model = load_model(model_file_path)
test_preds = model.predict(test_images)

In [ ]:
fig = plt.figure(figsize=(20,16))
for i in range(20):
    axis = fig.add_subplot(4, 5, i+1, xticks=[], yticks=[])
    plot_sample(test_images[i], test_preds[i], axis, "")
plt.show()

In [ ]:
feature_names = list(idlookup_df['FeatureName'])
image_ids = list(idlookup_df['ImageId']-1)
row_ids = list(idlookup_df['RowId'])

feature_list = []
for feature in feature_names:
    feature_list.append(feature_names.index(feature))

In [ ]:
predictions = []
for x,y in zip(image_ids, feature_list):
    predictions.append(test_preds[x][y])

In [ ]:
row_ids = pd.Series(row_ids, name = 'RowId')
locations = pd.Series(predictions, name = 'Location')
locations = locations.clip(0.0,96.0)
submission_result = pd.concat([row_ids,locations],axis = 1)
submission_result.to_csv('submission.csv',index = False)